In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import lifelines
import pycox
import xgboost as xgb
from sklearn.model_selection import train_test_split

from xgbse import XGBSEKaplanNeighbors, XGBSEKaplanTree, XGBSEDebiasedBCE, XGBSEBootstrapEstimator
from xgbse.converters import convert_data_to_xgb_format, convert_to_structured, convert_y
from xgbse.non_parametric import get_time_bins

from benchmark import BenchmarkLifelines, BenchmarkXGBoost, BenchmarkXGBSE, BenchmarkPysurvival
from pysurvival.models.survival_forest import ConditionalSurvivalForestModel

# setting seed
np.random.seed(42)

In [3]:
def split_train_test_valid(dataf, test_size=.2, valid_size=.01, random_state=1):
    df_train, df_test = train_test_split(dataf, test_size=test_size, random_state=random_state)
    df_train, df_valid = train_test_split(df_train, test_size=valid_size, random_state=random_state)
    return df_train, df_valid,  df_test

## Data

In [4]:
from pycox.datasets import rr_nl_nhp
df = rr_nl_nhp.read_df()
df.drop(["duration_true", "event_true", "censoring_true"], axis=1, inplace=True)
df_train, df_valid, df_test = split_train_test_valid(df)

T_train = df_train.duration
E_train = df_train.event
TIME_BINS = time_bins = get_time_bins(T_train, E_train, size=20)

## Model

Let us fit a model and check performance.

In [5]:
results = pd.DataFrame()

## Lifelines

In [6]:
bench_lifelines = BenchmarkLifelines(lifelines.CoxPHFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Cox-PH"
)

In [7]:
bench_lifelines.train()

In [8]:
cox_lifelines_results = bench_lifelines.test()
cox_lifelines_results

{'model': 'Cox-PH',
 'c-index': 0.7868823131406573,
 'ibs': 0.13529809371549895,
 'dcal_pval': 4.498777173708728e-43,
 'dcal_max_dev': 0.054902674947442935,
 'training_time': 2.3615710735321045,
 'inference_time': 0.011093854904174805}

In [9]:
results = results.append(cox_lifelines_results, ignore_index=True)

In [10]:
bench_lifelines = BenchmarkLifelines(lifelines.WeibullAFTFitter(),
        df_train.assign(duration = np.where(df_train.duration == 0, 1, df_train.duration)),
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Weibull AFT"
)

In [11]:
bench_lifelines.train()
weibull_aft_results = bench_lifelines.test()
weibull_aft_results

{'model': 'Weibull AFT',
 'c-index': 0.7868933380115156,
 'ibs': 0.13602422693625488,
 'dcal_pval': 2.880925330169667e-43,
 'dcal_max_dev': 0.05667257626985076,
 'training_time': 0.26163506507873535,
 'inference_time': 0.008216142654418945}

In [12]:
results = results.append(weibull_aft_results, ignore_index=True)

## XGBoost

In [13]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - AFT",
        "survival:aft",
)

In [14]:
bench_xgboost.train()
xgboost_aft_results = bench_xgboost.test()
xgboost_aft_results

{'model': 'XGB - AFT',
 'c-index': 0.8247420038146053,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.050727128982543945,
 'inference_time': 0.0007798671722412109}

In [15]:
results = results.append(xgboost_aft_results, ignore_index=True)

In [16]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - Cox",
        "survival:cox",
)
bench_xgboost.train()
xgboost_cox_results = bench_xgboost.test()
xgboost_cox_results

{'model': 'XGB - Cox',
 'c-index': 0.8256438268849773,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.0559542179107666,
 'inference_time': 0.0006079673767089844}

In [17]:
results = results.append(xgboost_cox_results, ignore_index=True)

## XGB SE

In [18]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEDebiasedBCE(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Debiased BCE",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bce_results = bench_xgboost_embedding.test()
xgboost_bce_results

{'model': 'XGBSE - Debiased BCE',
 'c-index': 0.8269204387353041,
 'ibs': 0.09752739549146452,
 'dcal_pval': 5.8911349042153576e-61,
 'dcal_max_dev': 0.04365250295331396,
 'training_time': 128.98045086860657,
 'inference_time': 0.9824099540710449}

In [19]:
results = results.append(xgboost_bce_results, ignore_index=True)

In [20]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanNeighbors(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Neighbors",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kn_results = bench_xgboost_embedding.test()
xgboost_kn_results

{'model': 'XGBSE - Kaplan Neighbors',
 'c-index': 0.8225066828334464,
 'ibs': 0.09971237872316244,
 'dcal_pval': 3.219039393014018e-30,
 'dcal_max_dev': 0.03427276372407928,
 'training_time': 112.96890211105347,
 'inference_time': 66.89054703712463}

In [21]:
results = results.append(xgboost_kn_results, ignore_index=True)

In [22]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanTree(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Tree",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kt_results = bench_xgboost_embedding.test()
xgboost_kt_results

{'model': 'XGBSE - Kaplan Tree',
 'c-index': 0.82121853464918,
 'ibs': 0.10129775993937155,
 'dcal_pval': 6.905144620707361e-44,
 'dcal_max_dev': 0.043777519983996954,
 'training_time': 0.5062479972839355,
 'inference_time': 0.004987955093383789}

In [23]:
results = results.append(xgboost_kt_results, ignore_index=True)

In [24]:
base_tree = XGBSEKaplanTree()

bench_xgboost_embedding = BenchmarkXGBSE(
    XGBSEBootstrapEstimator(base_tree, n_estimators=100),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Bootstrap Trees",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bootstrap_results = bench_xgboost_embedding.test()
xgboost_bootstrap_results

{'model': 'XGBSE - Bootstrap Trees',
 'c-index': 0.8261708611753944,
 'ibs': 0.09713889680012697,
 'dcal_pval': 1.4238032769395418e-34,
 'dcal_max_dev': 0.035288583245685845,
 'training_time': 40.94499111175537,
 'inference_time': 0.4194779396057129}

In [25]:
results = results.append(xgboost_bootstrap_results, ignore_index=True)

## Pysurvival

In [26]:
bench_pysurvival = BenchmarkPysurvival(ConditionalSurvivalForestModel(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Conditional Survival Forest",

)
bench_pysurvival.train()
pysurvival_results = bench_pysurvival.test()
pysurvival_results

{'model': 'Conditional Survival Forest',
 'c-index': 0.8107541136691464,
 'ibs': 0.11349372155633686,
 'dcal_pval': 1.8236116793086885e-108,
 'dcal_max_dev': 0.06696904963665812,
 'training_time': 17.024398803710938,
 'inference_time': 5786.263374090195}

In [27]:
results = results.append(pysurvival_results, ignore_index=True)

In [28]:
results.sort_values("c-index", ascending=False).round(3)

,c-index,dcal_max_dev,dcal_pval,ibs,inference_time,model,training_time
4,0.827,0.044,0.0,0.098,0.982,XGBSE - Debiased BCE,128.980
7,0.826,0.035,0.0,0.097,0.419,XGBSE - Bootstrap Trees,40.945
3,0.826,NaN,NaN,NaN,0.001,XGB - Cox,0.056
2,0.825,NaN,NaN,NaN,0.001,XGB - AFT,0.051
5,0.823,0.034,0.0,0.100,66.891,XGBSE - Kaplan Neighbors,112.969
6,0.821,0.044,0.0,0.101,0.005,XGBSE - Kaplan Tree,0.506
8,0.811,0.067,0.0,0.113,5786.263,Conditional Survival Forest,17.024
1,0.787,0.057,0.0,0.136,0.008,Weibull AFT,0.262
0,0.787,0.055,0.0,0.135,0.011,Cox-PH,2.362


In [29]:
results.sort_values("c-index", ascending=False).round(3).set_index('model').to_markdown()

'| model                       |   c-index |   dcal_max_dev |   dcal_pval |     ibs |   inference_time |   training_time |\n|:----------------------------|----------:|---------------:|------------:|--------:|-----------------:|----------------:|\n| XGBSE - Debiased BCE        |     0.827 |          0.044 |           0 |   0.098 |            0.982 |         128.98  |\n| XGBSE - Bootstrap Trees     |     0.826 |          0.035 |           0 |   0.097 |            0.419 |          40.945 |\n| XGB - Cox                   |     0.826 |        nan     |         nan | nan     |            0.001 |           0.056 |\n| XGB - AFT                   |     0.825 |        nan     |         nan | nan     |            0.001 |           0.051 |\n| XGBSE - Kaplan Neighbors    |     0.823 |          0.034 |           0 |   0.1   |           66.891 |         112.969 |\n| XGBSE - Kaplan Tree         |     0.821 |          0.044 |           0 |   0.101 |            0.005 |           0.506 |\n| Conditional S